# Projeto Final - Aprendizagem de Máquina 2025-2
**SAINT (Self-Attention and Intersample Transformer)**

Este notebook implementa todos os requisitos do projeto final, da segunda parte da disciplina de Aprendizagem de Máquina.

**Referência do SAINT:**
https://github.com/Actis92/lit-saint

## Instalando os pacotes

É necessário utilizar o `python 3.8` para instalar o SAINT corretamente. Aqui são instalados os seguintes pacotes:
- `lit-saint`: Implementação do modelo SAINT.
- `openml`: Biblioteca para carregar os datasets do OpenML.
- `autogluon`: Modelo de automl utilizado para comparação.
- `lightgbm`, ``xgboost`` e ``catboost``: Modelos para comparação.

In [ ]:
%pip install spacy==3.7.0 spacy-legacy==3.0.12 pandas==1.5.3 openml==0.14.2 \
    lit-saint==0.4.1 lightgbm==3.3.5 xgboost==1.7.6 catboost==1.2.5 autogluon==0.8.3

: 

## Configurando o ambiente

Faz as seguintes alterações:
1. Utiliza o estilo 'seaborn-v0_8-darkgrid' para os gráficos e 'husl' para a paleta de cores.
2. Ignora avisos para manter o notebook limpo.
3. Define uma semente aleatória para garantir reprodutibilidade.

In [ ]:
SEARCH_ITERATIONS = 20
RANDOM_STATE = 42
CV_FOLDS = 5
N_JOBS = -1

: 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import warnings

plt.style.use('seaborn-v0_8-darkgrid')
warnings.filterwarnings('ignore')
np.random.seed(RANDOM_STATE)

## Obtenção e preparação dos dados

Carrega os 2 menores datasets do OpenML para tarefas de classificação e regressão (valor ajustável no código).

In [ ]:
import openml

OPENML_CC18_ID = 99
NUM_DATASETS = 10

suite = openml.study.get_suite(suite_id=OPENML_CC18_ID)
datasets_df = openml.datasets.list_datasets(data_id=suite.data, output_format='dataframe')

# Seleciona os menores datasets
datasets_df_sorted = datasets_df.sort_values(by='NumberOfInstances')
top_datasets = datasets_df_sorted.head(NUM_DATASETS)

datasets_memory = {}
for idx, row in top_datasets.iterrows():
    dataset_id = row['did']
    dataset_name = row['name']
    print(f"Fetching {dataset_name} (ID: {dataset_id}, Instances: {row['NumberOfInstances']})...")
    
    try:
        dataset = openml.datasets.get_dataset(dataset_id)
        X, y, categorical_indicator, attribute_names = dataset.get_data(
            target=dataset.default_target_attribute,
            dataset_format='dataframe'
        )

        if y is not None:
            X['target'] = y
            
        datasets_memory[dataset_name] = X
    except Exception as e:
        print(f"Failed to load {dataset_name}: {e}")

print(f"Done! {len(datasets_memory)} datasets available in 'datasets_memory' dictionary.")

## Divisão dos dados em conjuntos de treinamento e teste

Cada dataset é dividido em:
- **Conjunto de treino**: 70%
- **Conjunto de teste**: 30%
- **Seed**: Definida para reprodutibilidade

In [ ]:
from sklearn.model_selection import train_test_split

train_test_splits = {}
for dataset_name, df in datasets_memory.items():
    X = df.drop(columns=['target'])
    y = df['target']
    
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.30, random_state=RANDOM_STATE
    )
    
    train_test_splits[dataset_name] = {
        'X_train': X_train,
        'X_test': X_test,
        'y_train': y_train,
        'y_test': y_test,
        'train_size': len(X_train),
        'test_size': len(X_test)
    }
    
    print(f"{dataset_name}:")
    print(f"  Treino: {len(X_train)} amostras (70%)")
    print(f"  Teste:  {len(X_test)} amostras (30%)")
    print()

print(f"Total de {len(train_test_splits)} datasets divididos com sucesso!")

## Busca de Hiperparâmetros com Validação Cruzada

Realiza a otimização de hiperparâmetros para os seguintes modelos usando RandomizedSearchCV com validação cruzada estratificada:
- **SAINT (lit-saint)**: Dimensionalidade, número de cabeças de atenção, taxa de aprendizado (implementação pendente)
- **LightGBM**: Número de folhas, profundidade, taxa de aprendizado
- **CatBoost**: Profundidade, iterações, taxa de aprendizado
- **XGBoost**: Profundidade, taxa de aprendizado, número de estimadores
- **AutoGluon**: AutoML com validação cruzada

> Os modelos clássicos salvam suas predições e metadados em `artifacts/<modelo>/<dataset_slug>/`. Nenhum objeto treinado é persistido.

In [ ]:
from artifact_utils import write_artifact_bundle, get_dataset_dir

MODEL_KEY_MAP = {
    'LightGBM': 'lightgbm',
    'XGBoost': 'xgboost',
    'CatBoost': 'catboost',
    'SAINT': 'saint',
    'AutoGluon': 'autogluon',
}


def artifact_exists(model_name: str, dataset_name: str) -> bool:
    """Check if metadata + predictions already exist for this dataset/model."""

    model_key = MODEL_KEY_MAP.get(model_name, model_name.lower())
    dataset_dir = get_dataset_dir(model_key, dataset_name)
    return (
        dataset_dir.joinpath('metadata.json').exists()
        and dataset_dir.joinpath('predictions.npz').exists()
    )


def artifact_output_dir(model_name: str, dataset_name: str) -> str:
    model_key = MODEL_KEY_MAP.get(model_name, model_name.lower())
    return str(get_dataset_dir(model_key, dataset_name))


### Definição do espaço amostral

Define o espaço de busca de hiperparâmetros para os seguintes modelos:
- **LightGBM**
- **CatBoost**
- **XGBoost**
- **SAINT (lit-saint)**

E define a função de preprocessamento dos dados e a função de busca de hiperparâmetros para os três primeiros modelos

In [ ]:
from sklearn.preprocessing import LabelEncoder, StandardScaler, OrdinalEncoder
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform, loguniform
from sklearn.metrics import accuracy_score
from catboost import CatBoostClassifier
from pytorch_lightning import Trainer
from joblib import parallel_backend
import lightgbm as lgb
import xgboost as xgb
import time

param_spaces_classifier = {
    'LightGBM': {
        'num_leaves': randint(20, 151),
        'max_depth': randint(3, 16),
        'learning_rate': loguniform(1e-2, 3e-1),
        'n_estimators': randint(50, 401),
        'min_child_samples': randint(5, 101),
        'subsample': uniform(0.5, 0.5),
        'colsample_bytree': uniform(0.5, 0.5),
        'reg_alpha': loguniform(1e-8, 10),
        'reg_lambda': loguniform(1e-8, 10)
    },
    'XGBoost': {
        'max_depth': randint(3, 16),
        'learning_rate': loguniform(1e-2, 3e-1),
        'n_estimators': randint(50, 401),
        'subsample': uniform(0.5, 0.5),
        'colsample_bytree': uniform(0.5, 0.5),
        'min_child_weight': randint(1, 11),
        'gamma': loguniform(1e-8, 1.0),
        'reg_alpha': loguniform(1e-8, 10),
        'reg_lambda': loguniform(1e-8, 10)
    },
    'CatBoost': {
        'depth': randint(3, 11),
        'border_count': randint(32, 256),
        'learning_rate': loguniform(1e-2, 3e-1),
        'iterations': randint(50, 401),
        'l2_leaf_reg': uniform(1, 9),
        'random_strength': uniform(0, 10),
        'bagging_temperature': uniform(0, 1)
    },
    "Saint": {
        'depth': randint(3, 11),
        'heads': randint(4, 9),
        'dropout': uniform(0.0, 0.5),
        'learning_rate': loguniform(1e-4, 1e-2),
        'batch_size': randint(64, 257),
        'epochs': randint(5, 21)
    }
}


def run_classification_searches(X_train, y_train, cv_strategy, target_models=None):
    results = {}
    search_space = {
        'LightGBM': (
            lgb.LGBMClassifier(random_state=RANDOM_STATE, verbose=-1),
            param_spaces_classifier['LightGBM'],
            N_JOBS,
            'accuracy'
        ),
        'XGBoost': (
            xgb.XGBClassifier(random_state=RANDOM_STATE, eval_metric='logloss', use_label_encoder=False),
            param_spaces_classifier['XGBoost'],
            N_JOBS,
            'accuracy'
        ),
        'CatBoost': (
            CatBoostClassifier(random_state=RANDOM_STATE, verbose=0),
            param_spaces_classifier['CatBoost'],
            1,
            'accuracy'
        )
    }

    target_set = set(target_models) if target_models else None

    for model_name, (estimator, param_dist, n_jobs, scoring) in search_space.items():
        if target_set and model_name not in target_set:
            continue
        print(f"\n  {model_name} (RandomizedSearchCV)...", end=' ', flush=True)

        start_time = time.time()
        try:
            search = RandomizedSearchCV(
                estimator=estimator,
                param_distributions=param_dist,
                n_iter=SEARCH_ITERATIONS,
                cv=cv_strategy,
                n_jobs=n_jobs,
                scoring=scoring,
                random_state=RANDOM_STATE,
                verbose=0
            )
            # Use thread-based parallelism to avoid pickling failures from loky processes.
            with parallel_backend('threading'):
                search.fit(X_train, y_train)
            end_time = time.time()

            results[model_name] = {
                'best_params': search.best_params_,
                'best_score': search.best_score_,
                'best_model': search.best_estimator_,
                'time': end_time - start_time
            }

            print(f"✓ Score: {search.best_score_:.4f} | Tempo: {end_time - start_time:.2f}s")
        except Exception as e:
            print(f"✗ Erro: {str(e)[:50]}")
            results[model_name] = None
    return results


def persist_tree_model_artifacts(model_name, dataset_name, processed_data, result):
    if result is None or result.get('best_model') is None:
        print(f"  ✗ {model_name} não retornou estimador treinado.")
        return

    estimator = result['best_model']
    X_test_encoded = processed_data['X_test_encoded']
    y_test_encoded = processed_data['y_test_encoded']
    label_encoder = processed_data['label_encoder']

    try:
        y_prob = estimator.predict_proba(X_test_encoded)
    except Exception:
        y_prob = None

    y_pred = estimator.predict(X_test_encoded)
    test_accuracy = accuracy_score(y_test_encoded, y_pred)

    model_key = MODEL_KEY_MAP.get(model_name, model_name.lower())
    write_artifact_bundle(
        model_key=model_key,
        dataset_name=dataset_name,
        y_true=y_test_encoded,
        y_pred=y_pred,
        y_prob=y_prob,
        class_labels=label_encoder.classes_.tolist(),
        metrics={
            'cv_accuracy': result.get('best_score'),
            'test_accuracy': test_accuracy,
        },
        hyperparams=result.get('best_params'),
        runtime_seconds=result.get('time'),
        extra_metadata={
            'train_samples': len(processed_data['X_train_encoded']),
            'test_samples': len(y_test_encoded),
        },
    )
    print(f"    → Artefatos salvos em {artifact_output_dir(model_name, dataset_name)}")

In [ ]:
from sklearn.model_selection import StratifiedKFold
import pandas as pd

def preprocess_dataset(splits):
    X_train = splits['X_train']
    y_train = splits['y_train']
    X_test = splits['X_test']
    y_test = splits['y_test']

    le = LabelEncoder()
    y_train_encoded = le.fit_transform(y_train)
    y_test_encoded = le.transform(y_test)

    X_train_encoded = X_train.copy()
    X_test_encoded = X_test.copy()
    categorical_cols = X_train_encoded.select_dtypes(include=['object', 'category']).columns.tolist()

    if categorical_cols:
        oe = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
        X_train_encoded[categorical_cols] = oe.fit_transform(X_train_encoded[categorical_cols].astype(str))
        X_test_encoded[categorical_cols] = oe.transform(X_test_encoded[categorical_cols].astype(str))

    scaler = StandardScaler()
    numeric_cols = X_train_encoded.select_dtypes(include=[np.number]).columns
    X_train_scaled = X_train_encoded.copy()
    X_test_scaled = X_test_encoded.copy()
    if len(numeric_cols) > 0:
        X_train_scaled[numeric_cols] = scaler.fit_transform(X_train_encoded[numeric_cols])
        X_test_scaled[numeric_cols] = scaler.transform(X_test_encoded[numeric_cols])

    return {
        'X_train_encoded': X_train_encoded,
        'X_test_encoded': X_test_encoded,
        'X_train_scaled': X_train_scaled,
        'X_test_scaled': X_test_scaled,
        'y_train_encoded': y_train_encoded,
        'y_test_encoded': y_test_encoded,
        'y_train_original': y_train.reset_index(drop=True),
        'y_test_original': y_test.reset_index(drop=True),
        'label_encoder': le,
        'categorical_cols': categorical_cols,
        'numeric_cols': numeric_cols
    }

# Dicionários para acompanhar resultados e dados processados
processed_datasets = {}
total_datasets = len(train_test_splits)
classical_artifacts = 0

# Busca para modelos baseados em árvores
for dataset_idx, (dataset_name, splits) in enumerate(train_test_splits.items(), 1):
    print(f"\n{'='*80}")
    print(f"Dataset {dataset_idx}/{total_datasets}: {dataset_name}")
    print(f"{'='*80}")

    processed_data = preprocess_dataset(splits)
    processed_datasets[dataset_name] = processed_data

    X_train_encoded = processed_data['X_train_encoded']
    y_train_encoded = processed_data['y_train_encoded']

    # Estratégia de validação cruzada
    min_class_count = pd.Series(y_train_encoded).value_counts().min()
    total_train_samples = len(X_train_encoded)
    n_splits = min(CV_FOLDS, min_class_count)
    cv_strategy = StratifiedKFold(
        n_splits=n_splits,
        shuffle=True,
        random_state=RANDOM_STATE
    )

    print(f"\n  Amostras: {total_train_samples} | Min classe: {min_class_count} | CV folds: {n_splits}")

    pending_models = [
        model
        for model in ['LightGBM', 'XGBoost', 'CatBoost']
        if not artifact_exists(model, dataset_name)
    ]

    if not pending_models:
        print("  ✓ Artefatos existentes para modelos clássicos. Nada a fazer.")
        continue

    print(f"\n  Executando busca para: {', '.join(pending_models)}")
    search_results = run_classification_searches(
        X_train=X_train_encoded,
        y_train=y_train_encoded,
        cv_strategy=cv_strategy,
        target_models=pending_models,
    )

    for model_name, result in search_results.items():
        if model_name in pending_models:
            persist_tree_model_artifacts(model_name, dataset_name, processed_data, result)
            if result is not None:
                classical_artifacts += 1

    print(f"\n  ✓ Busca concluída para modelos clássicos!")

print(f"\n{'='*80}")
print(f"Artefatos clássicos atualizados para {classical_artifacts} treinamentos.")
print(f"{'='*80}")


### Treinamento dedicado com SAINT
Nesta etapa executamos somente a busca randômica e o re-treinamento do SAINT, separando explicitamente seu pipeline dos demais modelos. As saídas (metadados + probabilidades) são gravadas em `artifacts/saint/<dataset_slug>/` para uso posterior pelo notebook de comparação.

In [ ]:
from lit_saint.config import NetworkConfig, TransformerConfig, TrainConfig, OptimizerConfig
from lit_saint import Saint, SaintConfig, SaintDatamodule, SaintTrainer
import torch

def prepare_saint_dataframe(X, y, validation_ratio=0.2, min_val_samples=10, random_state=None):
    """Cria o DataFrame esperado pelo SaintDatamodule com colunas target e split."""
    df = X.copy().reset_index(drop=True)
    df['target'] = y.reset_index(drop=True)
    df['split'] = 'train'

    if len(df) <= 1:
        return df

    val_size = max(int(len(df) * validation_ratio), min_val_samples)
    if val_size >= len(df):
        val_size = max(1, len(df) // 5) or 1

    val_idx = df.sample(n=val_size, random_state=random_state).index
    df.loc[val_idx, 'split'] = 'validation'
    return df


def extract_prediction_array(predictions):
    """Normaliza as saídas do SAINT para um array NumPy."""
    if isinstance(predictions, dict):
        for key in ('predictions', 'logits', 'probs'):
            if key in predictions:
                predictions = predictions[key]
                break
        else:
            for value in predictions.values():
                if hasattr(value, 'shape'):
                    predictions = value
                    break
            else:
                raise ValueError(f"Não foi possível encontrar previsões nas chaves: {list(predictions.keys())}")

    if hasattr(predictions, 'detach'):
        predictions = predictions.detach()
    if hasattr(predictions, 'cpu'):
        predictions = predictions.cpu()
    if hasattr(predictions, 'numpy'):
        predictions = predictions.numpy()
    return predictions


def saint_array_to_probabilities(preds_array):
    tensor = torch.tensor(preds_array).float()
    if tensor.ndim == 1 or (tensor.ndim == 2 and tensor.shape[1] == 1):
        probs = torch.sigmoid(tensor).numpy().flatten()
        return np.vstack([1 - probs, probs]).T
    return torch.softmax(tensor, dim=1).numpy()


def predictions_to_labels(predictions):
    predictions = np.asarray(predictions)
    if predictions.ndim == 1:
        return (predictions > 0.5).astype(int)
    return np.argmax(predictions, axis=1)


def run_saint_random_search(processed_data, max_trials=None):
    """Executa busca randômica de hiperparâmetros e reentreina o SAINT com os melhores valores."""
    start_time = time.time()
    trials = max_trials or SEARCH_ITERATIONS
    X_train_features = processed_data['X_train_scaled']
    y_train_raw = processed_data['y_train_original']
    y_train_encoded = processed_data['y_train_encoded']
    X_test_features = processed_data['X_test_scaled']
    y_test_encoded = processed_data['y_test_encoded']
    label_encoder = processed_data['label_encoder']

    df_train_full = prepare_saint_dataframe(
        X=X_train_features,
        y=y_train_raw,
        validation_ratio=0.2,
        min_val_samples=10,
        random_state=RANDOM_STATE
    )

    n_classes = len(np.unique(y_train_encoded))
    best_score = -np.inf
    best_params = None

    print("  SAINT (Random Search)...", end=' ', flush=True)

    for trial_idx in range(trials):
        params = {
            'depth': int(param_spaces_classifier['Saint']['depth'].rvs()),
            'heads': int(param_spaces_classifier['Saint']['heads'].rvs()),
            'dropout': float(param_spaces_classifier['Saint']['dropout'].rvs()),
            'learning_rate': float(param_spaces_classifier['Saint']['learning_rate'].rvs()),
            'batch_size': int(param_spaces_classifier['Saint']['batch_size'].rvs()),
            'epochs': int(param_spaces_classifier['Saint']['epochs'].rvs())
        }

        try:
            data_module = SaintDatamodule(
                df=df_train_full,
                target='target',
                split_column='split'
            )

            saint_config = SaintConfig(
                network=NetworkConfig(
                    transformer=TransformerConfig(
                        depth=params['depth'],
                        heads=params['heads'],
                        dropout=params['dropout']
                    )
                ),
                train=TrainConfig(
                    epochs=params['epochs'],
                    batch_size=params['batch_size'],
                    optimizer=OptimizerConfig(learning_rate=params['learning_rate'])
                )
            )

            saint_model = Saint(
                categories=data_module.categorical_dims,
                continuous=data_module.numerical_columns,
                config=saint_config,
                dim_target=n_classes
            )

            trainer = Trainer(
                max_epochs=params['epochs'],
                enable_progress_bar=False,
                logger=False,
                enable_model_summary=False
            )
            saint_trainer = SaintTrainer(trainer=trainer)
            saint_trainer.fit(model=saint_model, datamodule=data_module, enable_pretraining=False)

            df_val = df_train_full[df_train_full['split'] == 'validation'].copy()
            if df_val.empty:
                continue

            predictions = saint_trainer.predict(
                model=saint_model,
                df=df_val.drop(columns=['target', 'split']),
                datamodule=data_module
            )
            preds_array = extract_prediction_array(predictions)
            y_pred = predictions_to_labels(preds_array)
            y_true = label_encoder.transform(df_val['target'].values)
            score = accuracy_score(y_true, y_pred)

            if score > best_score:
                best_score = score
                best_params = params
        except Exception as e:
            print(f"\n    Erro SAINT (tentativa {trial_idx + 1}): {str(e)[:80]}")

    if best_params is None:
        print("✗ Não foi possível treinar SAINT.")
        return None

    # Reentreina utilizando todo o conjunto de treino, mantendo um pequeno holdout para o datamodule
    df_train_final = prepare_saint_dataframe(
        X=X_train_features,
        y=y_train_raw,
        validation_ratio=0.05,
        min_val_samples=1,
        random_state=RANDOM_STATE
    )

    data_module = SaintDatamodule(
        df=df_train_final,
        target='target',
        split_column='split'
    )

    saint_config = SaintConfig(
        network=NetworkConfig(
            transformer=TransformerConfig(
                depth=best_params['depth'],
                heads=best_params['heads'],
                dropout=best_params['dropout']
            )
        ),
        train=TrainConfig(
            epochs=best_params['epochs'],
            batch_size=best_params['batch_size'],
            optimizer=OptimizerConfig(learning_rate=best_params['learning_rate'])
        )
    )

    saint_model = Saint(
        categories=data_module.categorical_dims,
        continuous=data_module.numerical_columns,
        config=saint_config,
        dim_target=n_classes
    )

    trainer = Trainer(
        max_epochs=best_params['epochs'],
        enable_progress_bar=False,
        logger=False,
        enable_model_summary=False
    )
    saint_trainer = SaintTrainer(trainer=trainer)
    saint_trainer.fit(model=saint_model, datamodule=data_module, enable_pretraining=False)

    df_test = X_test_features.copy().reset_index(drop=True)
    predictions = saint_trainer.predict(model=saint_model, df=df_test, datamodule=data_module)
    preds_array = extract_prediction_array(predictions)
    y_prob_test = saint_array_to_probabilities(preds_array)
    y_pred_test = predictions_to_labels(preds_array)
    test_score = accuracy_score(y_test_encoded, y_pred_test)
    
    end_time = time.time()
    total_time = end_time - start_time

    print(f"✓ Melhor score: {best_score:.4f} | Teste: {test_score:.4f} | Tempo: {total_time:.2f}s")
    
    # Retorna apenas as informações necessárias para gerar os artefatos
    return {
        'best_params': best_params,
        'best_score': best_score,
        'test_score': test_score,
        'time': total_time,
        'y_pred': y_pred_test,
        'y_prob': y_prob_test,
        'label_encoder': label_encoder,
    }


In [ ]:
# Loop exclusivo para SAINT
saint_datasets_processed = 0
for dataset_idx, (dataset_name, splits) in enumerate(train_test_splits.items(), 1):
    print(f"\n{'-'*80}")
    print(f"SAINT | Dataset {dataset_idx}/{len(train_test_splits)}: {dataset_name}")
    print(f"{'-'*80}")

    if artifact_exists('SAINT', dataset_name):
        print("  ✓ Artefatos SAINT já existem. Pulando...")
        continue

    processed_data = processed_datasets.get(dataset_name)
    if processed_data is None:
        processed_data = preprocess_dataset(splits)
        processed_datasets[dataset_name] = processed_data

    saint_result = run_saint_random_search(processed_data)

    if saint_result is None:
        print("  ✗ SAINT não treinado.")
        continue

    write_artifact_bundle(
        model_key=MODEL_KEY_MAP['SAINT'],
        dataset_name=dataset_name,
        y_true=processed_data['y_test_encoded'],
        y_pred=saint_result['y_pred'],
        y_prob=saint_result['y_prob'],
        class_labels=processed_data['label_encoder'].classes_.tolist(),
        metrics={
            'cv_accuracy': saint_result['best_score'],
            'test_accuracy': saint_result['test_score'],
        },
        hyperparams=saint_result['best_params'],
        runtime_seconds=saint_result['time'],
        extra_metadata={
            'train_samples': len(processed_data['X_train_encoded']),
            'test_samples': len(processed_data['X_test_encoded']),
            'search_iterations': SEARCH_ITERATIONS,
        },
    )
    print(f"  ✓ Artefatos SAINT salvos em {artifact_output_dir('SAINT', dataset_name)}")
    saint_datasets_processed += 1

print(f"\n{'-'*80}")
print(f"SAINT executado para {saint_datasets_processed} datasets.")
print(f"{'-'*80}")


### Treinamento dedicado com AutoGluon
Nesta etapa executamos o AutoGluon AutoML separadamente, utilizando validação automática e ensemble de modelos. Assim como nos demais blocos, os resultados são persistidos em `artifacts/autogluon/<dataset_slug>/` para consumo offline.

In [ ]:
from autogluon.tabular import TabularPredictor
import shutil

autogluon_datasets_processed = 0
total_datasets = len(train_test_splits)
for dataset_idx, (dataset_name, splits) in enumerate(train_test_splits.items(), 1):
    print(f"\n{'-'*80}")
    print(f"AutoGluon | Dataset {dataset_idx}/{total_datasets}: {dataset_name}")
    print(f"{'-'*80}")

    if artifact_exists('AutoGluon', dataset_name):
        print("  ✓ Artefatos AutoGluon já existem. Pulando...")
        continue

    processed_data = processed_datasets.get(dataset_name)
    if processed_data is None:
        processed_data = preprocess_dataset(splits)
        processed_datasets[dataset_name] = processed_data

    X_train_encoded = processed_data['X_train_encoded']
    y_train_encoded = processed_data['y_train_encoded']
    X_test_encoded = processed_data['X_test_encoded']
    y_test_encoded = processed_data['y_test_encoded']

    print(f"  AutoGluon...", end=' ', flush=True)
    start_time = time.time()
    try:
        ag_dir = get_dataset_dir(MODEL_KEY_MAP['AutoGluon'], dataset_name) / 'autogluon_internal'
        if ag_dir.exists():
            shutil.rmtree(ag_dir)
        ag_dir.mkdir(parents=True, exist_ok=True)
        
        train_data = X_train_encoded.copy()
        train_data['target'] = y_train_encoded

        test_data = X_test_encoded.copy()
        test_data['target'] = y_test_encoded

        predictor = TabularPredictor(
            label='target',
            problem_type='multiclass',
            eval_metric='accuracy',
            path=str(ag_dir)
        )

        predictor.fit(
            train_data,
            time_limit=300,
            presets='best_quality'
        )

        leaderboard = predictor.leaderboard(test_data, silent=True)
        best_score_ag = leaderboard.iloc[0].get('score_test')
        if best_score_ag is None:
            best_score_ag = leaderboard.iloc[0].get('score_val')
        best_score_ag = float(best_score_ag) if best_score_ag is not None else np.nan
        
        y_prob = predictor.predict_proba(X_test_encoded).to_numpy()
        y_pred = predictor.predict(X_test_encoded)
        test_accuracy = accuracy_score(y_test_encoded, y_pred)
        
        total_time = time.time() - start_time

        write_artifact_bundle(
            model_key=MODEL_KEY_MAP['AutoGluon'],
            dataset_name=dataset_name,
            y_true=y_test_encoded,
            y_pred=y_pred,
            y_prob=y_prob,
            class_labels=processed_data['label_encoder'].classes_.tolist(),
            metrics={
                'leaderboard_score': best_score_ag,
                'test_accuracy': test_accuracy,
            },
            hyperparams={'time_limit': 300, 'presets': 'best_quality'},
            runtime_seconds=total_time,
            extra_metadata={
                'train_samples': len(X_train_encoded),
                'test_samples': len(y_test_encoded),
            },
        )

        print(f"✓ Score Eval: {best_score_ag:.4f} | Test: {test_accuracy:.4f} | Tempo: {total_time:.2f}s")
        print(f"    → Artefatos salvos em {artifact_output_dir('AutoGluon', dataset_name)}")
        autogluon_datasets_processed += 1
    except Exception as e:
        print(f"✗ Erro: {str(e)[:50]}")

print(f"\n{'-'*80}")
print(f"AutoGluon executado para {autogluon_datasets_processed} datasets.")
print(f"{'-'*80}")
